In [1]:
# check whether GPU is provided
!nvcc --version
!nvidia-smi

'nvcc' is not recognized as an internal or external command,
operable program or batch file.
'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
import subprocess

# Check CUDA version
cuda_version = subprocess.run(['nvcc', '--version'], capture_output=True, text=True)
print(cuda_version.stdout)

In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O cats_and_dogs_filtered.zip

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
import requests

url = "https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip"

response = requests.get(url, verify=False)

with open("cats_and_dogs_filtered.zip", "wb") as f:
    f.write(response.content)

c:\Users\mcoel\Documents\self_repos\Colab-AI-PucMinas\.venv\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'storage.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [5]:
import zipfile

local_zip = 'cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('data_cats_dogs/')
zip_ref.close()

In [7]:
import os
# import numpy as np
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

base_dir = 'data_cats_dogs/cats_and_dogs_filtered/'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
# test_dir = os.path.join(base_dir, 'test')

**Building our network**

In [8]:
from tensorflow.keras import layers
from tensorflow.keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

c:\Users\mcoel\Documents\self_repos\Colab-AI-PucMinas\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 15, 15, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     3,211,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,453,121 (13.17 MB)

 Trainable params: 3,453,121 (13.17 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
from tensorflow.keras import optimizers

# compile the model
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-4),
              metrics=['acc'])

**Data preprocessing**

Devemos converter os dados em tensores de ponto flutuante. Devemos seguir as seguintes etapas:

Leia os arquivos de imagem.
Decodifique em pixels RGB
Converta-os em tensores de ponto flutuante.
Rescale de cada pixels de (0 a 255) para [0, 1]

O keras.preprocessing.image contém a classe ImageDataGenerator, que permite configurar geradores Python que podem transformar automaticamente arquivos de imagem em disco em lotes de tensores pré-processados. 


In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


Vamos dar uma olhada no formato dos arquivos gerados pelo python

Foram produzidos batches de imagens RGB de 150x150 (shape (20, 150, 150, 3)) e labels binárias (shape (20,))

20 é o número de amostras em cada batch (o tamanho do batch)

In [12]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data batch shape: (20, 150, 150, 3)
labels batch shape: (20,)


**Treinando o modelo**

Vamos ajustar nosso modelo aos dados usando o gerador, usando o método **fit_generator**

train_generator: (primeiro argumento) produzirá batches indefinidamente. 

Como os dados estão sendo gerados infinitamente, o gerador precisa saber por exemplo quantas amostras extrair do gerador antes de declarar uma época.

Assim, usamos o **steps_per_epoch** para que, depois de extrair "steps_per_epoch" batches do gerador, o model.fit irá para a próxima época. Neste caso, cada batch possui 20 amostras, portanto, serão necesssário 100 batches até atingirmos o objetivo de 2000 amostras.  

**validation_data**: pode ser um gerador (como neste caso) e portanto, temos que fornecer o argumento **validation_steps** pois o gerador irá gerar dados infinitamente. O argumento validation_steps determina quantos batches serão extraídos do gerador Python de validação para a avaliação do modelo (50 pois nosso conjunto de validação possui 1000 amostras). 


In [14]:
history = model.fit(
      train_generator,
      steps_per_epoch=100,
      epochs=5,
      validation_data=validation_generator,
      validation_steps=50)

Epoch 1/5


c:\Users\mcoel\Documents\self_repos\Colab-AI-PucMinas\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


100/100 ━━━━━━━━━━━━━━━━━━━━ 51s 483ms/step - acc: 0.4974 - loss: 0.6975 - val_acc: 0.5250 - val_loss: 0.6843
Epoch 2/5


C:\Users\mcoel\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


AttributeError: 'NoneType' object has no attribute 'items'

In [ ]:
model.save('cats_and_dogs_small_1.h5')

Vamos plotar a loss e a acurácia do modelo sobre os dados de treinamento e validação durante o treinamento:


In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

Observe que a acurácia dos dados de treinamento aumenta linearmente ao longo do tempo até atingir quase 100% (e validação não passa de 70%).

A validation loss atinge seu mínimo depois de apenas cinco épocas, e a train loss continua diminuindo linearmente até atingir quase 0.

Como temos apenas poucas amostras de treinamento (2000), o maior risco é de overfitting (poderíamos resolver o problema com dropout). 

No entanto, usaremos uma técnica muito como para visão computacional que pode solucionar o problema de overfitting: **data augmentation**



Mas, antes de abordarmos o assunto de data augmentation, vamos aprender a realizar algumas poredições no nosso modelo treinado.

**Realizando Predições**

Obtendo uma imagem de exemplo usando no treinamento (veja que esta imagem já está pré-processada)

In [ ]:
data_batch, labels_batch = validation_generator[0]
print('data batch shape:', data_batch.shape)
print(data_batch[5].shape)

x = data_batch[5]

Observe o shape da imagem (150, 150, 3), para a predição temos que converter para um tensor 4D

In [ ]:
x = x.reshape(1, 150, 150, 3)

print (x.shape)

Realizando a inferência: a loss function binary_crossentropy retorna um valor de probabilidade entre 0 e 1 para a classificação binária (cat - 0 e dog - 1)

In [ ]:
prob = model.predict(x)

print(prob)

Abrindo uma imagem diretamente do diretório. Obsserve que agora a imagem não está pre-processada

In [ ]:
from google.colab.patches import cv2_imshow
import numpy as np
import cv2

path = os.path.join(base_dir, 'validation/dogs/dog.2010.jpg')

img = cv2.imread(path) 
cv2_imshow(img)

In [ ]:
print(img.shape)

Vamos carregar a imagem usando PIL (biblioteca para processamento de imagens em Python)

In [ ]:
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from PIL import Image

img = load_img(path, target_size=(150, 150))

Rescale de cada pixels de (0 a 255) para [0, 1]

In [ ]:
from numpy import asarray

img = asarray(img)

print('Data Type: %s' % img.dtype)
print('Min: %.3f, Max: %.3f' % (img.min(), img.max()))

img = img.astype('float32')
img /= 255.0

print('Min: %.3f, Max: %.3f' % (img.min(), img.max()))

In [ ]:
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

print(x.shape)

In [ ]:
prob = model.predict(x)

print(prob)

**Usando data augmentation**

O problema de overfitting foi causado por ter poucas amostras para aprender, ou seja, faz com que o modelo seja incapazes de generalizar para novos dados.

Data augmentation gera mais dados de treinamento a partir de amostras existentes de treinamento, por meio de várias transformações aleatórias que produzem novas imagens. 

O objetivo é que, durante o treinamento, nosso modelo nunca veja exatamente a mesma imagem duas vezes. Isso ajuda o modelo a se expor a mais aspectos dos dados e a generalizar melhor.

Você pude utilizar operações de processamento de imagens comuns para produzir novas amostras de dados a partir das imagens de treinamento.

Acesse para exemplos:

https://github.com/albumentations-team/albumentations

https://github.com/codebox/image_augmentor


No Keras, isso pode ser feito configurando várias transformações aleatórias a serem executadas nas imagens lidas pela nossa instância do ImageDataGenerator. 


In [ ]:
datagen = ImageDataGenerator(
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

Estas são apenas algumas das opções disponíveis (para mais, consulte a documentação do Keras). 

rotation_range é um valor em graus (0-180), um intervalo no qual gira aleatoriamente as imagens.

width_shift e height_shift são intervalos dentro dos quais rotaciona aleatoriamente imagens na vertical ou na horizontal.

shear_range é para aplicar aleatoriamente transformações de cisalhamento.

zoom_range é para ampliar aleatoriamente as imagens.

horizontal_flip é para virar aleatoriamente metade das imagens horizontalmente

fill_mode é a estratégia usada para preencher pixels recém-criados, que podem aparecer após uma rotação ou uma mudança de largura / altura.

In [ ]:
# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')

# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')

# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

In [ ]:
from keras.preprocessing import image

fnames = [os.path.join(train_cats_dir, fname) for fname in os.listdir(train_cats_dir)]

# We pick one image to "augment"
img_path = fnames[3]

# Read the image and resize it
img = image.load_img(img_path, target_size=(150, 150))

# Convert it to a Numpy array with shape (150, 150, 3)
x = image.img_to_array(img)

# Reshape it to (1, 150, 150, 3)
x = x.reshape((1,) + x.shape)

# The .flow() command below generates batches of randomly transformed images.
# It will loop indefinitely, so we need to `break` the loop at some point!
i = 0
for batch in datagen.flow(x, batch_size=1):
    plt.figure(i)
    imgplot = plt.imshow(image.array_to_img(batch[0]))
    i += 1
    if i % 4 == 0:
        break

plt.show()

Se treinarmos uma nova rede usando data augmentation, a rede nunca verá duas vezes a mesma entrada. No entanto, isso pode não ser suficiente para se livrar completamente do problema de overfitting. Para isso, também adicionaremos uma camada Dropout ao modelo.

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-4),
              metrics=['acc'])

Treinando o modelo usando data augmentation e dropout



In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

# Note that the validation data should not be augmented!
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=50)

In [ ]:
model.save('cats_and_dogs_small_2.h5')

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

Observe que agora não estamos mais enfrentando o problema de overfitting. 

As curvas de treinamento estão acompanhando de perto as curvas de validação.

Referência: François Chollet. Deep Learning with Python. November 2017  